In [ ]:
from google.colab import drive
import pandas as pd
import pickle

# Mount Google Drive
drive.mount('/content/drive')

# Path to data folder
data_path = "/content/drive/MyDrive/3DCV/HW2/Q2/data/"

# List of pickle files
files = ["images.pkl", "point_desc.pkl", "points3D.pkl", "train.pkl"]



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **P3p + RANSAC from scratch**

Image #164 (valid_img10.jpg) — matches: 3580

Rotation (Rodrigues): [0.0552  ...  0.995   ...  0.1425]

Translation: [-3.4498  ...  -0.8014  ...   4.45  ]



In [43]:
import numpy as np
import pickle
import cv2
from scipy.spatial.transform import Rotation as R

# --- Load data ---
# data_path = "/content/drive/MyDrive/3DCV/HW2/Q2/data/"
# with open(data_path + "images.pkl", "rb") as f:
#     images_df = pickle.load(f)
# with open(data_path + "point_desc.pkl", "rb") as f:
#     point_desc_df = pickle.load(f)
# with open(data_path + "points3D.pkl", "rb") as f:
#     points3D_df = pickle.load(f)

# --- Camera intrinsics and distortion ---
K = np.array([[1868.27, 0, 540],
              [0, 1869.18, 960],
              [0, 0, 1]], dtype=np.float64)
distCoeffs = np.array([[0.0847023, -0.192929, -0.000201144, -0.000725352]], dtype=np.float64)

# --- Build lookup for 3D points ---
point3D_map = {pid: xyz for pid, xyz in zip(points3D_df["POINT_ID"], points3D_df["XYZ"])}

# --- P3P solver (simplified using SVD) ---
def p3p_solver(pts_3D, bearings):
    centroid_3D = np.mean(pts_3D, axis=0)
    P_centered = pts_3D - centroid_3D

    centroid_bear = np.mean(bearings, axis=0)
    U_centered = bearings - centroid_bear

    H = U_centered.T @ P_centered
    U_svd, S, Vt = np.linalg.svd(H)
    R_est = Vt.T @ U_svd.T
    if np.linalg.det(R_est) < 0:
        Vt[-1, :] *= -1
        R_est = Vt.T @ U_svd.T

    t_est = centroid_3D - R_est @ centroid_bear
    return R_est, t_est

# --- Get validation images ---
val_images = images_df[images_df["IMAGE_ID"] > 163]

poses = []

for img_row in val_images.itertuples():
    img_id = img_row.IMAGE_ID
    img_name = img_row.NAME
    print(f"Processing Image ID={img_id}, Name={img_name}")

    # --- Get 2D-3D correspondences ---
    pts_rows = point_desc_df[point_desc_df["IMAGE_ID"] == img_id]
    pts_rows = pts_rows[pts_rows["POINT_ID"] != -1]

    if len(pts_rows) < 6:  # too few points
        continue

    pts_2D = np.stack(pts_rows["XY"].values).astype(np.float64)
    pts_3D = np.array([point3D_map[pid] for pid in pts_rows["POINT_ID"]], dtype=np.float64)

    # --- Undistort points and compute bearings ---
    pts_2D_undist = cv2.undistortPoints(
        pts_2D.reshape(-1, 1, 2), K, distCoeffs
    ).reshape(-1, 2)
    bearings = np.hstack([pts_2D_undist, np.ones((len(pts_2D_undist), 1))])
    bearings /= np.linalg.norm(bearings, axis=1, keepdims=True)

    # --- RANSAC ---
    n_iter = 200
    best_inliers = 0
    best_R = None
    best_t = None
    threshold_angle = 5  # degrees

    for _ in range(n_iter):
        idx = np.random.choice(len(pts_3D), 3, replace=False)
        R_try, t_try = p3p_solver(pts_3D[idx], bearings[idx])

        proj = (R_try @ pts_3D.T + t_try[:, np.newaxis]).T
        proj /= np.linalg.norm(proj, axis=1, keepdims=True)
        cos_sim = np.sum(proj * bearings, axis=1)
        inliers = np.sum(cos_sim > np.cos(np.deg2rad(threshold_angle)))

        if inliers > best_inliers:
            best_inliers = inliers
            best_R = R_try
            best_t = t_try

    if best_R is not None:

        rvec, _ = cv2.Rodrigues(best_R)
        print("R = ", rvec.T,"     t = ", best_t)
        poses.append((img_id, img_name, rvec.ravel(), best_t, best_inliers))

# --- Compute translation and rotation errors ---
trans_errors = []
rot_errors = []

for img_id, img_name, rvec_est, tvec_est, n_inliers in poses:
    gt_row = images_df[images_df["IMAGE_ID"] == img_id]
    if gt_row.empty:
        continue
    gt_row = gt_row.iloc[0]
    t_gt = np.array([gt_row.TX, gt_row.TY, gt_row.TZ], dtype=np.float64)

    # translation error
    t_err = np.linalg.norm(tvec_est - t_gt)
    trans_errors.append(t_err)

    # rotation error
    q_gt = np.array([gt_row.QW, gt_row.QX, gt_row.QY, gt_row.QZ], dtype=np.float64)
    R_gt = R.from_quat([q_gt[1], q_gt[2], q_gt[3], q_gt[0]]).as_matrix()
    R_est = cv2.Rodrigues(rvec_est)[0]
    R_diff = R.from_matrix(R_est @ R_gt.T)
    angle_err = np.degrees(np.linalg.norm(R_diff.as_rotvec()))
    rot_errors.append(angle_err)

# --- Print median errors ---
median_trans_error = np.median(trans_errors)
median_rot_error = np.median(rot_errors)*3.14/180

print(f"\nMedian translation error: {median_trans_error:.4f}")
print(f"Median rotation error: {median_rot_error:.4f} degrees")


Processing Image ID=164, Name=valid_img10.jpg
Processing Image ID=165, Name=valid_img100.jpg
R =  [[-0.07463316 -1.6116841   0.09964732]]      t =  [ 3.39176763 -0.42580698  1.9342243 ]
Processing Image ID=166, Name=valid_img105.jpg
R =  [[-0.94980925 -1.20714323 -0.46258651]]      t =  [ 1.84928547 -4.13922497  8.06545227]
Processing Image ID=167, Name=valid_img110.jpg
R =  [[-1.11026301 -1.03086851 -0.1141686 ]]      t =  [ 1.39417556 -4.20719086  7.6326883 ]
Processing Image ID=168, Name=valid_img115.jpg
R =  [[-0.05146592 -1.98516062 -0.70755623]]      t =  [ 4.09741264 -0.35790278  2.55925203]
Processing Image ID=169, Name=valid_img120.jpg
R =  [[-0.26877901 -1.65784164 -0.61271998]]      t =  [ 2.97814866 -0.85634446  2.0616095 ]
Processing Image ID=170, Name=valid_img125.jpg
R =  [[-0.93883171 -1.31364091  0.01814206]]      t =  [ 1.98690309 -3.947148    7.98855677]
Processing Image ID=171, Name=valid_img130.jpg
R =  [[-1.00145793 -1.21177825 -0.04094775]]      t =  [ 1.7523761 